**THE CONSIDITION 2019 - EXAMPLE SOLUTION**

This notebook serves as a simple starting point for **The Considition 2019** programming competition. It is a modified version of https://nbviewer.jupyter.org/github/fastai/course-v3/blob/master/nbs/dl1/lesson3-camvid.ipynb, from the free fast.ai course *Practical Deep Learning for Coders, v3*.

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.callbacks.hooks import *

In [ ]:
# download training set from https://theconsidition.se/rules and insert path here
path = Path('/PATH/TO/TRAINING_DATASET') 

In [ ]:
# You might need to pre-process masks here for the model to correctly interpret/classify them
# Any other pre-processing of the dataset you want to do should also go here

In [ ]:
# set paths to training images and masks
path_lbl = path/'Masks/all'
path_img = path/'Images'

In [ ]:
# list paths to some training images
fnames = get_image_files(path_img)
fnames[:3]

In [ ]:
# list paths to some training masks
lbl_names = get_image_files(path_lbl)
lbl_names[:3]

In [ ]:
# show a training image
img_f = fnames[0]
img = open_image(img_f)
img.show(figsize=(5,5))

In [ ]:
# helper function to get the corresponding mask path of a training image
get_y_fn = lambda x: path_lbl/f'{x.stem}.png'

In [ ]:
# show the mask
mask = open_mask(get_y_fn(img_f))
mask.show(figsize=(5,5), alpha=1)

In [ ]:
# create an indexed list, where the index of each label corresponds to the pixel values of the masks
codelist = ['Background', 'Water', 'Building', 'Road']
codes = np.asarray(codelist); codes

In [ ]:
# create the data object
data = (SegmentationItemList.from_folder(path_img)
        .split_by_rand_pct()
        .label_from_func(get_y_fn, classes=codes)
        .transform(get_transforms(), size=128)
        .databunch(bs=4, num_workers=0)
        .normalize(imagenet_stats))

In [ ]:
# show some masks superimposed on their respective images
data.show_batch(2, figsize=(10,7))

In [ ]:
data.show_batch(2, figsize=(10,7), ds_type=DatasetType.Valid)

In [ ]:
# create a learner using the Unet architecture. Model pretrained on ImageNet
learn = unet_learner(data, models.resnet18, wd=1e-2, metrics=accuracy_thresh)

In [ ]:
# show some predictions from your current model, next to the ground-truth masks
learn.show_results()

In [ ]:
# TODO: Your model is now ready to be trained on your training dataset
# to further improve its performance for the competition!

In [ ]:
# save your model's current state
learn.save(path/'model')

In [ ]:
# export your learner/model, so it can be used for inference in other applications
learn.export(path/'model.pkl')